<img src="https://i.imgur.com/6U6q5jQ.png"/>

# Data Organization


Let me get the data on dengue from [Peru](https://www.datosabiertos.gob.pe/dataset/vigilancia-epidemiol%C3%B3gica-de-dengue):

In [73]:
import pandas as pd
linkData="https://github.com/SocialAnalytics-StrategicIntelligence/OrganizeExploreAndQuery/raw/main/dataFiles/dengue_ok.pkl"
dengue = pd.read_pickle(linkData)
dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398943 entries, 0 to 398942
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   departamento  398943 non-null  object        
 1   provincia     398943 non-null  object        
 2   distrito      398943 non-null  object        
 3   ano           398943 non-null  int64         
 4   semana        398943 non-null  int64         
 5   sexo          398943 non-null  object        
 6   edad          398943 non-null  int64         
 7   enfermedad    398943 non-null  category      
 8   year          398931 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(3), object(4)
memory usage: 24.7+ MB


In [74]:
dengue.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,ano,semana,edad,year
count,398943.00000,398943.00000,398943.00000,398931.00000
mean,2015.06170,22.61685,29.97476,.5f
min,2000.00000,1.00000,0.00000,.5f
25%,2011.00000,11.00000,15.00000,.5f
50%,2016.00000,19.00000,27.00000,.5f
75%,2020.00000,34.00000,42.00000,.5f
max,2022.00000,53.00000,106.00000,.5f
std,6.14862,14.89333,18.53260,nan


Each row is a person:

In [75]:
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,year
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,2000-01-01
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,2000-01-01
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,2000-01-01
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,2000-01-01
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,2000-01-01


If we wanted to count people, creating a column of ones helps:

In [76]:
dengue=dengue.assign(case=1)
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,year,case
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,2000-01-01,1
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,2000-01-01,1
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,2000-01-01,1
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,2000-01-01,1
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,2000-01-01,1


Let's start creating _data from these data_!

## Aggregation

In [242]:
indexList=['ano','enfermedad']
aggregator={'edad': ['mean','median'], 'case':['sum']}
ByYear_stats=dengue.groupby(indexList,observed=True).agg(aggregator)
ByYear_stats

edad          case
                       mean median    sum
ano  enfermedad                          
2000 SIN_SEÑALES  29.508788   27.0   4324
2001 SIN_SEÑALES  30.634282   28.0  15851
     GRAVE        31.572614   28.0    241
2002 SIN_SEÑALES  26.960178   24.0   6278
     ALARMA       12.000000   12.0      1
     GRAVE        21.928571   19.0     14
2003 SIN_SEÑALES  28.947719   27.0   2850
     GRAVE        38.000000   30.0     15
2004 SIN_SEÑALES  28.863269   26.0   7928
     GRAVE        23.794118   19.0     34
2005 SIN_SEÑALES  26.891200   24.0   5000
     GRAVE        23.533333   23.0     15
2006 SIN_SEÑALES  27.436258   25.0   3624
     GRAVE        13.500000    8.5      4
2007 SIN_SEÑALES  28.676543   26.0   5299
     GRAVE        34.285714   30.0     35
2008 SIN_SEÑALES  28.941008   26.0  10493
     GRAVE        27.342857   22.0     35
2009 SIN_SEÑALES  30.180729   28.0  11271
     GRAVE        17.727273   13.0     11
2010 SIN_SEÑALES  31.388572   29.0  12286
     ALARMA       30.573643   27.0    129
     GRAVE        29.512821   30.0     39
2011 SIN_SEÑALES  29.501838   27.0  12241
     ALARMA       26.409993   23.0   2822
     GRAVE        28.167630   25.0    173
2012 SIN_SEÑALES  28.204686   25.0  16645
     ALARMA       25.604320   21.0   3935
     GRAVE        25.220513   23.0    195
2013 SIN_SEÑALES  28.359894   26.0   8672
     ALARMA       26.481650   22.0   3297
     GRAVE        29.680556   25.0     72
2014 SIN_SEÑALES  29.084037   26.0  11495
     ALARMA       26.754755   22.0   3943
     GRAVE        29.652174   24.5     92
2015 SIN_SEÑALES  31.623656   29.0  23436
     ALARMA       31.460900   28.0   5908
     GRAVE        35.691057   32.0    123
2016 SIN_SEÑALES  31.109387   28.0  19463
     ALARMA       30.369930   27.0   3279
     GRAVE        32.048780   30.0    123
2017 SIN_SEÑALES  33.303543   30.0  36555
     ALARMA       33.212251   30.0   6285
     GRAVE        39.649351   35.0    231
2018 SIN_SEÑALES  30.047100   28.0   3397
     ALARMA       30.305785   28.0   1089
     GRAVE        30.250000   28.5     64
2019 SIN_SEÑALES  29.390619   27.0  10276
     ALARMA       29.883060   27.0   2745
     GRAVE        34.215190   31.5    158
2020 SIN_SEÑALES  29.400147   27.0  34040
     ALARMA       27.129586   23.0   5479
     GRAVE        30.840000   26.0    225
2021 SIN_SEÑALES  29.483831   27.0  36674
     ALARMA       28.555706   26.0   3698
     GRAVE        32.682171   27.0    129
2022 SIN_SEÑALES  30.288617   27.0  48625
     ALARMA       28.289552   25.0   7370
     GRAVE        35.146226   32.0    212

In [243]:
ByYear_stats.columns

MultiIndex([('edad',   'mean'),
            ('edad', 'median'),
            ('case',    'sum')],
           )

In [236]:
["_".join(name) for name in ByYear_stats.columns]

['edad_mean', 'edad_median', 'case_sum']

In [237]:
ByYear_stats.columns=["_".join(name) for name in ByYear_stats.columns]
ByYear_stats

edad_mean  edad_median  case_sum
departamento provincia  ano  enfermedad                                   
AMAZONAS     BAGUA      2000 SIN_SEÑALES  32.241860         31.0       215
                        2001 SIN_SEÑALES  30.109920         28.0       373
                        2002 SIN_SEÑALES  28.250000         25.0        20
                        2003 SIN_SEÑALES  30.317073         29.5        82
                        2004 SIN_SEÑALES  31.482759         32.0       116
...                                             ...          ...       ...
UCAYALI      PADRE ABAD 2022 ALARMA       27.126437         24.0        87
                             GRAVE        31.000000         31.0         2
             PURUS      2020 SIN_SEÑALES  46.000000         46.0         1
                        2022 SIN_SEÑALES  43.000000         43.0         1
                             ALARMA       17.000000         17.0         1

[2300 rows x 3 columns]

In [238]:
ByYear_stats.reset_index(drop=False,inplace=True)
ByYear_stats

,departamento,provincia,ano,enfermedad,edad_mean,edad_median,case_sum
0,AMAZONAS,BAGUA,2000,SIN_SEÑALES,32.241860,31.0,215
1,AMAZONAS,BAGUA,2001,SIN_SEÑALES,30.109920,28.0,373
2,AMAZONAS,BAGUA,2002,SIN_SEÑALES,28.250000,25.0,20
3,AMAZONAS,BAGUA,2003,SIN_SEÑALES,30.317073,29.5,82
4,AMAZONAS,BAGUA,2004,SIN_SEÑALES,31.482759,32.0,116
...,...,...,...,...,...,...,...
2295,UCAYALI,PADRE ABAD,2022,ALARMA,27.126437,24.0,87
2296,UCAYALI,PADRE ABAD,2022,GRAVE,31.000000,31.0,2
2297,UCAYALI,PURUS,2020,SIN_SEÑALES,46.000000,46.0,1
2298,UCAYALI,PURUS,2022,SIN_SEÑALES,43.000000,43.0,1


In [239]:
ByYear_stats.enfermedad.dtype

CategoricalDtype(categories=['SIN_SEÑALES', 'ALARMA', 'GRAVE'], ordered=True, categories_dtype=object)

In [240]:
ByYear_stats.set_index(['departamento','provincia','ano','enfermedad']).stack().reset_index()

,departamento,provincia,ano,enfermedad,level_4,0
0,AMAZONAS,BAGUA,2000,SIN_SEÑALES,edad_mean,32.24186
1,AMAZONAS,BAGUA,2000,SIN_SEÑALES,edad_median,31.00000
2,AMAZONAS,BAGUA,2000,SIN_SEÑALES,case_sum,215.00000
3,AMAZONAS,BAGUA,2001,SIN_SEÑALES,edad_mean,30.10992
4,AMAZONAS,BAGUA,2001,SIN_SEÑALES,edad_median,28.00000
...,...,...,...,...,...,...
6895,UCAYALI,PURUS,2022,SIN_SEÑALES,edad_median,43.00000
6896,UCAYALI,PURUS,2022,SIN_SEÑALES,case_sum,1.00000
6897,UCAYALI,PURUS,2022,ALARMA,edad_mean,17.00000
6898,UCAYALI,PURUS,2022,ALARMA,edad_median,17.00000


In [213]:
indexList=['departamento', 'provincia', 'ano', 'semana','enfermedad']
aggregator={'edad': ['mean','median'], 'case':['sum']}
ByProvinceWeek_stats=dengue.groupby(indexList,observed=True).agg(aggregator)
ByProvinceWeek_stats

edad        case
                                                      mean median  sum
departamento provincia  ano  semana enfermedad                        
AMAZONAS     BAGUA      2000 18     SIN_SEÑALES  34.352941   32.0   17
                             19     SIN_SEÑALES  29.825000   27.0   40
                             20     SIN_SEÑALES  35.172414   33.5   58
                             21     SIN_SEÑALES  37.444444   37.0   27
                             22     SIN_SEÑALES  30.708333   30.5   24
...                                                    ...    ...  ...
UCAYALI      PADRE ABAD 2022 52     SIN_SEÑALES  30.800000   29.0    5
                                    ALARMA       33.500000   33.5    2
             PURUS      2020 51     SIN_SEÑALES  46.000000   46.0    1
                        2022 28     ALARMA       17.000000   17.0    1
                             51     SIN_SEÑALES  43.000000   43.0    1

[36422 rows x 3 columns]

Notice:

In [214]:
ByProvinceWeek_stats.columns

MultiIndex([('edad',   'mean'),
            ('edad', 'median'),
            ('case',    'sum')],
           )

We may turn that multi-index structure in a simpler way:

In [215]:
ByProvinceWeek_stats.columns=['meanAge','medianAge','sumCases']
ByProvinceWeek_stats

meanAge  medianAge  \
departamento provincia  ano  semana enfermedad                          
AMAZONAS     BAGUA      2000 18     SIN_SEÑALES  34.352941       32.0   
                             19     SIN_SEÑALES  29.825000       27.0   
                             20     SIN_SEÑALES  35.172414       33.5   
                             21     SIN_SEÑALES  37.444444       37.0   
                             22     SIN_SEÑALES  30.708333       30.5   
...                                                    ...        ...   
UCAYALI      PADRE ABAD 2022 52     SIN_SEÑALES  30.800000       29.0   
                                    ALARMA       33.500000       33.5   
             PURUS      2020 51     SIN_SEÑALES  46.000000       46.0   
                        2022 28     ALARMA       17.000000       17.0   
                             51     SIN_SEÑALES  43.000000       43.0   

                                                 sumCases  
departamento provincia  ano  semana enfermedad             
AMAZONAS     BAGUA      2000 18     SIN_SEÑALES        17  
                             19     SIN_SEÑALES        40  
                             20     SIN_SEÑALES        58  
                             21     SIN_SEÑALES        27  
                             22     SIN_SEÑALES        24  
...                                                   ...  
UCAYALI      PADRE ABAD 2022 52     SIN_SEÑALES         5  
                                    ALARMA              2  
             PURUS      2020 51     SIN_SEÑALES         1  
                        2022 28     ALARMA              1  
                             51     SIN_SEÑALES         1  

[36422 rows x 3 columns]

In [216]:
ByProvinceWeek_stats.reset_index(drop=False,inplace=True)

ByProvinceWeek_stats

,departamento,provincia,ano,semana,enfermedad,meanAge,medianAge,sumCases
0,AMAZONAS,BAGUA,2000,18,SIN_SEÑALES,34.352941,32.0,17
1,AMAZONAS,BAGUA,2000,19,SIN_SEÑALES,29.825000,27.0,40
2,AMAZONAS,BAGUA,2000,20,SIN_SEÑALES,35.172414,33.5,58
3,AMAZONAS,BAGUA,2000,21,SIN_SEÑALES,37.444444,37.0,27
4,AMAZONAS,BAGUA,2000,22,SIN_SEÑALES,30.708333,30.5,24
...,...,...,...,...,...,...,...,...
36417,UCAYALI,PADRE ABAD,2022,52,SIN_SEÑALES,30.800000,29.0,5
36418,UCAYALI,PADRE ABAD,2022,52,ALARMA,33.500000,33.5,2
36419,UCAYALI,PURUS,2020,51,SIN_SEÑALES,46.000000,46.0,1
36420,UCAYALI,PURUS,2022,28,ALARMA,17.000000,17.0,1


In [217]:
ByProvinceWeek_stats.enfermedad.dtype

CategoricalDtype(categories=['SIN_SEÑALES', 'ALARMA', 'GRAVE'], ordered=True, categories_dtype=object)

In [218]:
ByProvinceWeek_stats.to_pickle('dataFiles/ByProvinceWeek_stats.pkl')

## Reshaping

### From Long to Wide

The object *CasesByWeek* shows the values in one column, and the other column serve as identifier (an index). Let's make a simple wide version (one index):

In [219]:
ByProvinceWeek_stats_Wide=ByProvinceWeek_stats.pivot_table(index=['departamento','provincia','ano','semana'],
                                                                 columns='enfermedad')
ByProvinceWeek_stats

,departamento,provincia,ano,semana,enfermedad,meanAge,medianAge,sumCases
0,AMAZONAS,BAGUA,2000,18,SIN_SEÑALES,34.352941,32.0,17
1,AMAZONAS,BAGUA,2000,19,SIN_SEÑALES,29.825000,27.0,40
2,AMAZONAS,BAGUA,2000,20,SIN_SEÑALES,35.172414,33.5,58
3,AMAZONAS,BAGUA,2000,21,SIN_SEÑALES,37.444444,37.0,27
4,AMAZONAS,BAGUA,2000,22,SIN_SEÑALES,30.708333,30.5,24
...,...,...,...,...,...,...,...,...
36417,UCAYALI,PADRE ABAD,2022,52,SIN_SEÑALES,30.800000,29.0,5
36418,UCAYALI,PADRE ABAD,2022,52,ALARMA,33.500000,33.5,2
36419,UCAYALI,PURUS,2020,51,SIN_SEÑALES,46.000000,46.0,1
36420,UCAYALI,PURUS,2022,28,ALARMA,17.000000,17.0,1


Simpler structure:

In [220]:
ByProvinceWeek_stats_Wide_flatten=ByProvinceWeek_stats.pivot_table(index=['departamento','provincia','ano','semana'],
                            columns='enfermedad').reset_index(drop=False)
ByProvinceWeek_stats_Wide_flatten

departamento   provincia   ano semana     meanAge               \
enfermedad                                       SIN_SEÑALES ALARMA GRAVE   
0              AMAZONAS       BAGUA  2000     18   34.352941    NaN   NaN   
1              AMAZONAS       BAGUA  2000     19   29.825000    NaN   NaN   
2              AMAZONAS       BAGUA  2000     20   35.172414    NaN   NaN   
3              AMAZONAS       BAGUA  2000     21   37.444444    NaN   NaN   
4              AMAZONAS       BAGUA  2000     22   30.708333    NaN   NaN   
...                 ...         ...   ...    ...         ...    ...   ...   
27198           UCAYALI  PADRE ABAD  2022     51   16.111111   17.0   NaN   
27199           UCAYALI  PADRE ABAD  2022     52   30.800000   33.5   NaN   
27200           UCAYALI       PURUS  2020     51   46.000000    NaN   NaN   
27201           UCAYALI       PURUS  2022     28         NaN   17.0   NaN   
27202           UCAYALI       PURUS  2022     51   43.000000    NaN   NaN   

             medianAge                 sumCases               
enfermedad SIN_SEÑALES ALARMA GRAVE SIN_SEÑALES ALARMA GRAVE  
0                 32.0    NaN   NaN        17.0    NaN   NaN  
1                 27.0    NaN   NaN        40.0    NaN   NaN  
2                 33.5    NaN   NaN        58.0    NaN   NaN  
3                 37.0    NaN   NaN        27.0    NaN   NaN  
4                 30.5    NaN   NaN        24.0    NaN   NaN  
...                ...    ...   ...         ...    ...   ...  
27198             17.0   17.0   NaN         9.0    1.0   NaN  
27199             29.0   33.5   NaN         5.0    2.0   NaN  
27200             46.0    NaN   NaN         1.0    NaN   NaN  
27201              NaN   17.0   NaN         NaN    1.0   NaN  
27202             43.0    NaN   NaN         1.0    NaN   NaN  

[27203 rows x 13 columns]

We have multi index, let's flatten them:

In [221]:
ByProvinceWeek_stats_Wide_flatten.columns

MultiIndex([('departamento',            ''),
            (   'provincia',            ''),
            (         'ano',            ''),
            (      'semana',            ''),
            (     'meanAge', 'SIN_SEÑALES'),
            (     'meanAge',      'ALARMA'),
            (     'meanAge',       'GRAVE'),
            (   'medianAge', 'SIN_SEÑALES'),
            (   'medianAge',      'ALARMA'),
            (   'medianAge',       'GRAVE'),
            (    'sumCases', 'SIN_SEÑALES'),
            (    'sumCases',      'ALARMA'),
            (    'sumCases',       'GRAVE')],
           names=[None, 'enfermedad'])

In [222]:
ByProvinceWeek_stats_Wide_flatten.columns.to_flat_index()

Index([        ('departamento', ''),            ('provincia', ''),
                        ('ano', ''),               ('semana', ''),
         ('meanAge', 'SIN_SEÑALES'),        ('meanAge', 'ALARMA'),
               ('meanAge', 'GRAVE'), ('medianAge', 'SIN_SEÑALES'),
            ('medianAge', 'ALARMA'),       ('medianAge', 'GRAVE'),
        ('sumCases', 'SIN_SEÑALES'),       ('sumCases', 'ALARMA'),
              ('sumCases', 'GRAVE')],
      dtype='object')

In [223]:
["_".join(a) if a[1]!='' else a[0] for a in ByProvinceWeek_stats_Wide_flatten.columns.to_flat_index()]

['departamento',
 'provincia',
 'ano',
 'semana',
 'meanAge_SIN_SEÑALES',
 'meanAge_ALARMA',
 'meanAge_GRAVE',
 'medianAge_SIN_SEÑALES',
 'medianAge_ALARMA',
 'medianAge_GRAVE',
 'sumCases_SIN_SEÑALES',
 'sumCases_ALARMA',
 'sumCases_GRAVE']

In [224]:
ByProvinceWeek_stats_Wide_flatten.columns=["_".join(a) if a[1]!='' else a[0] for a in ByProvinceWeek_stats_Wide_flatten.columns.to_flat_index()]
ByProvinceWeek_stats_Wide_flatten

,departamento,provincia,ano,semana,meanAge_SIN_SEÑALES,meanAge_ALARMA,meanAge_GRAVE,medianAge_SIN_SEÑALES,medianAge_ALARMA,medianAge_GRAVE,sumCases_SIN_SEÑALES,sumCases_ALARMA,sumCases_GRAVE
0,AMAZONAS,BAGUA,2000,18,34.352941,NaN,NaN,32.0,NaN,NaN,17.0,NaN,NaN
1,AMAZONAS,BAGUA,2000,19,29.825000,NaN,NaN,27.0,NaN,NaN,40.0,NaN,NaN
2,AMAZONAS,BAGUA,2000,20,35.172414,NaN,NaN,33.5,NaN,NaN,58.0,NaN,NaN
3,AMAZONAS,BAGUA,2000,21,37.444444,NaN,NaN,37.0,NaN,NaN,27.0,NaN,NaN
4,AMAZONAS,BAGUA,2000,22,30.708333,NaN,NaN,30.5,NaN,NaN,24.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27198,UCAYALI,PADRE ABAD,2022,51,16.111111,17.0,NaN,17.0,17.0,NaN,9.0,1.0,NaN
27199,UCAYALI,PADRE ABAD,2022,52,30.800000,33.5,NaN,29.0,33.5,NaN,5.0,2.0,NaN
27200,UCAYALI,PURUS,2020,51,46.000000,NaN,NaN,46.0,NaN,NaN,1.0,NaN,NaN
27201,UCAYALI,PURUS,2022,28,NaN,17.0,NaN,NaN,17.0,NaN,NaN,1.0,NaN


In [225]:
ByProvinceWeek_stats_Wide_flatten.to_csv('dataFiles/ByProvinceWeek_stats_Wide_flatten.csv',index=False)

### Wide to Long

We should be able to transfor this wide version into a long one:

In [226]:
ByProvinceWeek_stats_Wide.stack()

meanAge  medianAge  \
departamento provincia  ano  semana enfermedad                          
AMAZONAS     BAGUA      2000 18     SIN_SEÑALES  34.352941       32.0   
                             19     SIN_SEÑALES  29.825000       27.0   
                             20     SIN_SEÑALES  35.172414       33.5   
                             21     SIN_SEÑALES  37.444444       37.0   
                             22     SIN_SEÑALES  30.708333       30.5   
...                                                    ...        ...   
UCAYALI      PADRE ABAD 2022 52     SIN_SEÑALES  30.800000       29.0   
                                    ALARMA       33.500000       33.5   
             PURUS      2020 51     SIN_SEÑALES  46.000000       46.0   
                        2022 28     ALARMA       17.000000       17.0   
                             51     SIN_SEÑALES  43.000000       43.0   

                                                 sumCases  
departamento provincia  ano  semana enfermedad             
AMAZONAS     BAGUA      2000 18     SIN_SEÑALES      17.0  
                             19     SIN_SEÑALES      40.0  
                             20     SIN_SEÑALES      58.0  
                             21     SIN_SEÑALES      27.0  
                             22     SIN_SEÑALES      24.0  
...                                                   ...  
UCAYALI      PADRE ABAD 2022 52     SIN_SEÑALES       5.0  
                                    ALARMA            2.0  
             PURUS      2020 51     SIN_SEÑALES       1.0  
                        2022 28     ALARMA            1.0  
                             51     SIN_SEÑALES       1.0  

[36422 rows x 3 columns]

In [228]:
ByProvinceWeek_stats_Long=ByProvinceWeek_stats_Wide_flatten.set_index(['departamento','provincia','ano','semana']).stack().reset_index()
ByProvinceWeek_stats_Long

,departamento,provincia,ano,semana,level_4,0
0,AMAZONAS,BAGUA,2000,18,meanAge_SIN_SEÑALES,34.352941
1,AMAZONAS,BAGUA,2000,18,medianAge_SIN_SEÑALES,32.000000
2,AMAZONAS,BAGUA,2000,18,sumCases_SIN_SEÑALES,17.000000
3,AMAZONAS,BAGUA,2000,19,meanAge_SIN_SEÑALES,29.825000
4,AMAZONAS,BAGUA,2000,19,medianAge_SIN_SEÑALES,27.000000
...,...,...,...,...,...,...
109261,UCAYALI,PURUS,2022,28,medianAge_ALARMA,17.000000
109262,UCAYALI,PURUS,2022,28,sumCases_ALARMA,1.000000
109263,UCAYALI,PURUS,2022,51,meanAge_SIN_SEÑALES,43.000000
109264,UCAYALI,PURUS,2022,51,medianAge_SIN_SEÑALES,43.000000


In [229]:
ByProvinceWeek_stats_Long.rename(columns={'level_4':'variable',0:'value'},inplace=True)
ByProvinceWeek_stats_Long

,departamento,provincia,ano,semana,variable,value
0,AMAZONAS,BAGUA,2000,18,meanAge_SIN_SEÑALES,34.352941
1,AMAZONAS,BAGUA,2000,18,medianAge_SIN_SEÑALES,32.000000
2,AMAZONAS,BAGUA,2000,18,sumCases_SIN_SEÑALES,17.000000
3,AMAZONAS,BAGUA,2000,19,meanAge_SIN_SEÑALES,29.825000
4,AMAZONAS,BAGUA,2000,19,medianAge_SIN_SEÑALES,27.000000
...,...,...,...,...,...,...
109261,UCAYALI,PURUS,2022,28,medianAge_ALARMA,17.000000
109262,UCAYALI,PURUS,2022,28,sumCases_ALARMA,1.000000
109263,UCAYALI,PURUS,2022,51,meanAge_SIN_SEÑALES,43.000000
109264,UCAYALI,PURUS,2022,51,medianAge_SIN_SEÑALES,43.000000


In [230]:
ByProvinceWeek_stats_Long.info() # enfermedad is object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109266 entries, 0 to 109265
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   departamento  109266 non-null  object 
 1   provincia     109266 non-null  object 
 2   ano           109266 non-null  int64  
 3   semana        109266 non-null  int64  
 4   variable      109266 non-null  object 
 5   value         109266 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 5.0+ MB


In [231]:
ByProvinceWeek_stats_Long.variable.dtype

dtype('O')

In [232]:
ByProvinceWeek_stats_Wide.stack().reset_index().enfermedad.dtype

CategoricalDtype(categories=['SIN_SEÑALES', 'ALARMA', 'GRAVE'], ordered=True, categories_dtype=object)